In [1]:
#Import Dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#Read CSV from provided Cloud URl
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Preprocessing Steps

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns=["EIN", "NAME"])

In [3]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff= 200
application_types_to_replace=application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts()< cutoff].index.tolist()
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classy=application_df['CLASSIFICATION'].value_counts()
classy


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1248        1
C6100        1
C1820        1
C1900        1
C2150        1
Name: count, Length: 71, dtype: int64

In [6]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff= 1000
classifications_to_replace=classy[classy <cutoff].index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df=pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])

Base Model with 72.76% accuracy

In [8]:
# Split our preprocessed data into our features and target arrays
y=application_df['IS_SUCCESSFUL']
x=application_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.25, random_state=42)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(x_train)

# Scale the data
X_train_scaled = X_scaler.transform(x_train)
X_test_scaled = X_scaler.transform(x_test)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
creature_features=x_train.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=creature_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

d:\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
#Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
check_it= tf.keras.callbacks.ModelCheckpoint(
    filepath='model_weights_{epoch:02d}.weights.h5',
    save_weights_only=True,
    save_freq=5 * len(X_train_scaled)
)

In [12]:
# Train the model
drip=nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 626us/step - accuracy: 0.7131 - loss: 0.5836
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 602us/step - accuracy: 0.7247 - loss: 0.5572
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - accuracy: 0.7270 - loss: 0.5538
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - accuracy: 0.7299 - loss: 0.5505
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step - accuracy: 0.7274 - loss: 0.5522
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - accuracy: 0.7252 - loss: 0.5535
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - accuracy: 0.7288 - loss: 0.5490
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step - accuracy: 0.7338 - loss: 0.5444
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step - accuracy: 0.7361 - loss: 0.5435
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 605us/step - accuracy: 0.7388 - loss: 0.5412
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - accuracy: 0.7341 - loss: 0.5414
Epoch 12/100
804/80

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 910us/step - accuracy: 0.7276 - loss: 0.5634
Loss: 0.5634266138076782, Accuracy: 0.727580189704895


Attempt 1- Reduced Neurons in Layers Results: 72.52% Accuracy, .24% decrease from base model

In [14]:
# Define the model - deep neural net for Attempt 1.
creature_features=x_train.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation='relu', input_dim=creature_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 40)             │         1,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           410 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,181 (8.52 KB)

 Trainable params: 2,181 (8.52 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
#Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
check_it= tf.keras.callbacks.ModelCheckpoint(
    filepath='model_weights_{epoch:02d}.weights.h5',
    save_weights_only=True,
    save_freq=5 * len(X_train_scaled)
)

In [16]:
# Train the model
drip=nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 607us/step - accuracy: 0.6859 - loss: 0.6237
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 616us/step - accuracy: 0.7235 - loss: 0.5593
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step - accuracy: 0.7298 - loss: 0.5546
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 588us/step - accuracy: 0.7291 - loss: 0.5531
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - accuracy: 0.7307 - loss: 0.5529
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - accuracy: 0.7307 - loss: 0.5497
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - accuracy: 0.7319 - loss: 0.5460
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - accuracy: 0.7284 - loss: 0.5492
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - accuracy: 0.7299 - loss: 0.5506
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step - accuracy: 0.7377 - loss: 0.5437
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - accuracy: 0.7375 - loss: 0.5403
Epoch 12/100
804/80

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 788us/step - accuracy: 0.7252 - loss: 0.5581
Loss: 0.5581188797950745, Accuracy: 0.7252478003501892


Attempt 2- Double Neurons and Epochs in Model Results: 73.19% Accuracy, .43% higher than base model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
creature_features=x_train.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=160, activation='relu', input_dim=creature_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 160)            │         7,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 60)             │         9,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            61 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,761 (65.47 KB)

 Trainable params: 16,761 (65.47 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
#Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
check_it= tf.keras.callbacks.ModelCheckpoint(
    filepath='model_weights_{epoch:02d}.weights.h5',
    save_weights_only=True,
    save_freq=5 * len(X_train_scaled)
)

In [20]:
# Train the model
drip=nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 669us/step - accuracy: 0.7096 - loss: 0.5832
Epoch 2/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 664us/step - accuracy: 0.7205 - loss: 0.5628
Epoch 3/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 671us/step - accuracy: 0.7320 - loss: 0.5495
Epoch 4/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 660us/step - accuracy: 0.7347 - loss: 0.5475
Epoch 5/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 651us/step - accuracy: 0.7290 - loss: 0.5517
Epoch 6/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 663us/step - accuracy: 0.7284 - loss: 0.5506
Epoch 7/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 676us/step - accuracy: 0.7298 - loss: 0.5511
Epoch 8/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 680us/step - accuracy: 0.7317 - loss: 0.5445
Epoch 9/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 712us/step - accuracy: 0.7357 - loss: 0.5457
Epoch 10/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 692us/step - accuracy: 0.7325 - loss: 0.5429
Epoch 11/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 686us/step - accuracy: 0.7316 - loss: 0.5466
Epoch 12/200
804/80

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 785us/step - accuracy: 0.7319 - loss: 0.5856
Loss: 0.5856025815010071, Accuracy: 0.7318950295448303


Attempt 3 - Double Neuron amounts, Added learning rate at .001 and changed activation to tanH Result: 72.87% accuracy, .11% higher than Base Model

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
creature_features=x_train.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=160, activation='tanh', input_dim=creature_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 160)            │         7,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 60)             │         9,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            61 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,761 (65.47 KB)

 Trainable params: 16,761 (65.47 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
#Compile the model
learning_rate= 0.001
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
nn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
check_it= tf.keras.callbacks.ModelCheckpoint(
    filepath='model_weights_{epoch:02d}.weights.h5',
    save_weights_only=True,
    save_freq=5 * len(X_train_scaled)
)

In [24]:
# Train the model
drip=nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 665us/step - accuracy: 0.7123 - loss: 0.5842
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 685us/step - accuracy: 0.7237 - loss: 0.5611
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 709us/step - accuracy: 0.7302 - loss: 0.5487
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 695us/step - accuracy: 0.7328 - loss: 0.5480
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 672us/step - accuracy: 0.7308 - loss: 0.5496
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step - accuracy: 0.7350 - loss: 0.5456
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 688us/step - accuracy: 0.7293 - loss: 0.5508
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 695us/step - accuracy: 0.7351 - loss: 0.5454
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 668us/step - accuracy: 0.7350 - loss: 0.5476
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 629us/step - accuracy: 0.7343 - loss: 0.5443
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 651us/step - accuracy: 0.7404 - loss: 0.5392
Epoch 12/100
804/80

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 817us/step - accuracy: 0.7287 - loss: 0.5650
Loss: 0.564973771572113, Accuracy: 0.7287463545799255


In [28]:
nn.save('AlphabetSoupCharity_Optimization.h5')